In [ ]:
import functools
import gc
import itertools
import os
from itertools import chain
import datetime

import anndata as ad
import faiss
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from os.path import join
from matplotlib import rcParams
from sklearn.preprocessing import Normalizer
from sklearn.utils.extmath import randomized_svd

import scglue

In [ ]:
os.environ['CUDA_VISIBLE_DEVICE'] = '1'

In [ ]:
PATH = "./cache/HF-50K-ATLAS"
os.makedirs(PATH, exist_ok=True)

In [ ]:
atac_all = ad.read_h5ad("/home/yanxh/data/HumanFetal/ATAC/adata_atac_raw.h5ad")  # moved to /home/sda1/yanxh/data/HFA
rna_all = ad.read_h5ad("/home/yanxh/data/HumanFetal/RNA/adata_rna_raw.h5ad")

In [ ]:
data_root = '/home/yanxh/data/HumanFetal_50k'

atac0 = sc.read_h5ad(join(data_root, 'ATAC/adata_atac.h5ad'))
rna0 = sc.read_h5ad(join(data_root, 'RNA/adata_rna_sampled.h5ad'))

In [ ]:
atac = atac_all[atac0.obs_names].copy()
atac.obs['cell_type'] = atac0.obs.cell_type.values
atac.obs['domain'] = 'atac'
atac.obs['tissue'] = ['cerebrum' if _.lower()=='brain' else _ for _ in atac0.obs.tissue.values]  # rename 'brain' as 'cerebrum'
atac.obs['tissue']

rna = rna_all[rna0.obs_names].copy()
rna.obs['cell_type'] = rna0.obs.Main_cluster_name.values
rna.obs['domain'] = 'rna'
# rna.obs = rna.obs[['domain', 'cell_type', 'Organ']].copy()

rna.var_names = rna.var.gene_short_name.values.astype('str')
rna = rna[:, ~rna.var_names.duplicated()]

del atac0, rna0
gc.collect()

### process feature names

In [ ]:
scglue.data.get_gene_annotation(
    rna, gtf="/home/yanxh/data/HumanFetal/gencode.v42.chr_patch_hapl_scaff.annotation.gtf",
    gtf_by="gene_name"
)
# exclude NaN rows
rna = rna[:, pd.notna(rna.var["chromStart"])].copy()

# rna.var.loc[:, ["chrom", "chromStart", "chromEnd"]].head()

split = atac.var_names.str.split(r"[:-]")
atac.var["chrom"] = split.map(lambda x: x[0])
atac.var["chromStart"] = split.map(lambda x: x[1]).astype(int)
atac.var["chromEnd"] = split.map(lambda x: x[2]).astype(int)
atac.var.head()

### Organ balancing

In [ ]:
rna_organ_fracs = rna.obs["Organ"].str.lower().value_counts() / rna.shape[0]
atac_organ_fracs = atac.obs["tissue"].str.lower().value_counts() / atac.shape[0]
cmp_organ_fracs = pd.DataFrame({"rna": rna_organ_fracs, "atac": atac_organ_fracs})

organ_min_fracs = cmp_organ_fracs.min(axis=1)

In [ ]:
rs = np.random.RandomState(0)
rna_subidx, atac_subidx = [], []
for organ, min_frac in organ_min_fracs.iteritems():
    print(f"Dealing with {organ}...")
    rna_idx = np.where(rna.obs["Organ"].str.lower() == organ)[0]
    rna_subidx.append(rs.choice(rna_idx, max(1, round(min_frac * rna.shape[0])), replace=False))
    atac_idx = np.where(atac.obs["tissue"].str.lower() == organ)[0]
    atac_subidx.append(rs.choice(atac_idx, max(1, round(min_frac * atac.shape[0])), replace=False))
rna_subidx = np.concatenate(rna_subidx)
rna_mask = np.zeros(rna.shape[0], dtype=bool)
rna_mask[rna_subidx] = True
rna.obs["mask"] = rna_mask
atac_subidx = np.concatenate(atac_subidx)
atac_mask = np.zeros(atac.shape[0], dtype=bool)
atac_mask[atac_subidx] = True
atac.obs["mask"] = atac_mask

In [ ]:
rna_organ_balancing = np.sqrt(cmp_organ_fracs["atac"] / cmp_organ_fracs["rna"])
atac_organ_balancing = np.sqrt(cmp_organ_fracs["rna"] / cmp_organ_fracs["atac"])

In [ ]:
rna.obs["organ_balancing"] = rna_organ_balancing.loc[rna.obs["Organ"].str.lower()].to_numpy()
atac.obs["organ_balancing"] = atac_organ_balancing.loc[atac.obs["tissue"].str.lower()].to_numpy()

### Gene selection and Dimreduction

In [ ]:
hvg_df = sc.pp.highly_variable_genes(rna[rna.obs["mask"], :], n_top_genes=4000, flavor="seurat_v3", inplace=False)
rna.var = rna.var.assign(**hvg_df.to_dict(orient="series"))

In [ ]:
rna.layers["raw_counts"] = rna.X.copy()
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
rna = rna[:, rna.var.highly_variable].copy()
gc.collect()

In [ ]:
X = rna.X
X_masked = X[rna.obs["mask"]]
mean = X_masked.mean(axis=0).A1
std = np.sqrt(X_masked.power(2).mean(axis=0).A1 - mean ** 2)
X = X.toarray()
X -= mean
X /= std
X = X.clip(-10, 10)
X_masked = X[rna.obs["mask"]]

In [ ]:
u, s, vh = randomized_svd(X_masked.T @ X_masked, 100, n_iter=15, random_state=0)
rna.obsm["X_pca"] = X @ vh.T

In [ ]:
rna.X = rna.layers["raw_counts"]
del rna.layers["raw_counts"], X, X_masked, mean, std, u, s, vh
gc.collect()

In [ ]:
X = scglue.num.tfidf(atac.X)
X = Normalizer(norm="l1").fit_transform(X)
X = np.log1p(X * 1e4)
X_masked = X[atac.obs["mask"]]
u, s, vh = randomized_svd(X_masked, 100, n_iter=15, random_state=0)
X_lsi = X @ vh.T / s
X_lsi -= X_lsi.mean(axis=1, keepdims=True)
X_lsi /= X_lsi.std(axis=1, ddof=1, keepdims=True)

del X, X_masked, X_lsi, u, s, vh
gc.collect()

# np.save('./cache/HF-50K-ATLAS/x_lsi.npy', atac.obsm["X_lsi"])
# X_lsi = np.load('./cache/HF-50K-ATLAS/x_lsi.npy')

atac.obsm["X_lsi"] = X_lsi.astype(np.float32)

### Creat graph

generate hvg for atac

In [ ]:
start_time = datetime.datetime.now()

In [ ]:
guidance = scglue.genomics.rna_anchored_guidance_graph(rna, atac)
scglue.graph.check_graph(guidance, [rna, atac])

# 2. scGLUE Pretrain

In [ ]:
scglue.models.configure_dataset(
    rna, "NB", use_highly_variable=True,
    use_rep="X_pca",       
    use_dsc_weight="organ_balancing",
#     use_cell_type='cell_type'  # does not help
)

In [ ]:
scglue.models.configure_dataset(
    atac, "NB", use_highly_variable=True,
    use_rep="X_lsi",
    use_dsc_weight="organ_balancing"
)

In [ ]:
guidance_hvf = guidance.subgraph(chain(
    rna.var.query("highly_variable").index,
    atac.var.query("highly_variable").index
)).copy()

# nx.write_graphml(guidance, f"{PATH}/full.graphml.gz")
# nx.write_graphml(guidance_hvf, f"{PATH}/sub.graphml.gz")

# graph = nx.read_graphml(f"{PATH}/full.graphml.gz")
# subgraph = nx.read_graphml(f"{PATH}/sub.graphml.gz")

In [ ]:
glue = scglue.models.fit_SCGLUE(
    {"rna": rna, "atac": atac}, guidance_hvf,
    fit_kws={"directory": PATH}
)

In [ ]:
end_time = datetime.datetime.now()
train_time = (end_time - start_time).total_seconds()
print('training time, ', train_time)

In [ ]:
glue.save(f"{PATH}/glue.dill")
# glue = scglue.models.load_model("glue.dill")

# Label transfer

In [ ]:
start_time = datetime.datetime.now()

In [ ]:
rna.obsm["X_glue"] = glue.encode_data("rna", rna)
atac.obsm["X_glue"] = glue.encode_data("atac", atac)

In [ ]:
atac.obs['cell_type_bkp'] = atac.obs['cell_type'].values 
atac.obs = atac.obs.drop(columns=['cell_type'])

In [ ]:
scglue.data.transfer_labels(rna, atac, "cell_type", use_rep="X_glue")

In [ ]:
end_time = datetime.datetime.now()
test_time = (end_time - start_time).total_seconds()
print('test time, ', test_time)

In [ ]:
train_time + test_time

In [ ]:
(atac.obs.cell_type == atac.obs.cell_type_bkp).mean()